# Generative Adversarial Networks - Implementation from scratch
---

> **For Generating Handwritten Digits**

## Importing Dependencies:

In [1]:
import torch
from torch import nn

import math
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms

In [2]:
# good practice to do, so that the experiment can be replicated identically on any machine
torch.manual_seed(111)

---

## Choosing CPU/GPU:

In [3]:
# If GPU is available than will use GPU else continue with CPU

device = ""
if torch.cuda.is_available():
    device = torch.devie("cuda")
else:
    device = torch.device("cpu")
    
device

device(type='cpu')

---
## Cooking the Data for Training:

In [4]:
transform = transforms.Compose(
[transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

# .ToTensor() - converts data to a pyTorch Tensor
# .Normalize() - converts the range of the tensor Coefficients

---
### LOADING the data from torchvision datasets

In [5]:
train_dataset = torchvision.datasets.MNIST(
root=".", train=True, download=True, transform=transform
)

---
### Creating the data loader:

In [6]:
batch_sz = 32
train_dataLoader = torch.utils.data.DataLoader(
train_dataset, batch_sz, shuffle=True
)

---
### Data Visualization:

In [7]:
real_sample_set, mnist_labels = next(iter(train_dataLoader))

#for i in range(16):
#    axis = plt.subplot(5,5,i+1)
#   plt.imshow(real_sample_set[i].reshape(28,28))
#    plt.xticks([])
#    plt.yticks([])


---

# Discriminator and Generator Implementation:

### Discriminator:

In [8]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        x = x.view(x.size(0), 784)
        output = self.model(x)
        return output

In [9]:
# Calling Discriminator model to the object & also selecting CPU/GPU

discriminator_ = Discriminator().to(device=device)

### Generator:

In [10]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            
            nn.Linear(256, 512),
            nn.ReLU(),
            
            nn.Linear(512, 1024),
            nn.ReLU(),
            
            nn.Linear(1024, 784),
            nn.Tanh(),
        )
        
    def forward(self, x):
        output = self.model(x)
        output = output.view(x.size(0), 1, 28, 28)
        return output

In [11]:
generator_ = Generator().to(device=device)

---

## Training the Models:

In [12]:
# Defining the Params

learning_rate = 0.0001
epochs_cnt = 50
loss_function = nn.BCELoss()

# Setting up Optimizer for both the Neural Networks
optimizer_discriminator = torch.optim.Adam(discriminator_.parameters(), lr=learning_rate)
optimizer_generator = torch.optim.Adam(generator_.parameters(), lr=learning_rate)

In [13]:
import time

In [14]:
# Training using the Loop:

start = time.time()
for epoch in range(epochs_cnt):
    for n, (real_sample_set, mnist_labels) in enumerate(train_dataLoader):
        
        # Sending the data to device for training Discriminator
        real_sample_set = real_sample_set.to(device=device)
        real_sample_labels = torch.ones((batch_sz, 1)).to(device=device)
        latent_space_samples = torch.randn((batch_sz, 100)).to(device=device)
        
        generated_samples = generator_(latent_space_samples)
        generated_samples_labels = torch.zeros((batch_sz,1)).to(device=device)
        
        all_samples = torch.cat((real_sample_set, generated_samples))
        all_samples_labels = torch.cat((real_sample_labels, generated_samples_labels))
        
        # Training the Discriminator
        discriminator_.zero_grad()
        output_discriminator = discriminator_(all_samples)
        loss_discriminator = loss_function(output_discriminator, all_samples_labels)
        
        loss_discriminator.backward()
        optimizer_discriminator.step()
        
        # Sending the data to device for training Generator
        latent_space_samples = torch.randn((batch_sz, 100)).to(device=device)
        
        # Training the Generator
        generator_.zero_grad()
        generated_samples = generator_(latent_space_samples)
        output_discriminator_generated = discriminator_(generated_samples)
        loss_generator = loss_function(output_discriminator_generated, real_sample_labels)
        loss_generator.backward()
        optimizer_generator.step()
        
        end = time.time()
        # Show loss
        if n == batch_sz-1:
            print("Epoch: {} Loss D.: {}".format(epoch, loss_discriminator))
            print("Epoch: {} Loss G.: {}".format(epoch, loss_generator))
            print("------------------------ Time Elapsed: {} seconds ------------------------".format(end-start))
        

Epoch: 0 Loss D.: 0.5205891132354736
Epoch: 0 Loss G.: 0.5769919753074646
------------------------ Time Elapsed: 1.6337950229644775 seconds ------------------------
Epoch: 1 Loss D.: 0.04461481049656868
Epoch: 1 Loss G.: 5.167757987976074
------------------------ Time Elapsed: 116.05373883247375 seconds ------------------------
Epoch: 2 Loss D.: 0.018259791657328606
Epoch: 2 Loss G.: 6.160158634185791
------------------------ Time Elapsed: 247.20123100280762 seconds ------------------------
Epoch: 3 Loss D.: 0.0280546136200428
Epoch: 3 Loss G.: 10.527824401855469
------------------------ Time Elapsed: 383.5022203922272 seconds ------------------------
Epoch: 4 Loss D.: 0.0018875167006626725
Epoch: 4 Loss G.: 9.207801818847656
------------------------ Time Elapsed: 524.6139590740204 seconds ------------------------
Epoch: 5 Loss D.: 0.01979658752679825
Epoch: 5 Loss G.: 4.719379901885986
------------------------ Time Elapsed: 664.0207996368408 seconds ------------------------
Epoch: 6 L

---

## Generated Samples Verifying:

In [29]:
latent_space_samples = torch.randn(batch_sz, 100).to(device=device)
generated_samples = generator_(latent_space_samples)
print(generated_samples[:2])

tensor([[[[-0.9987, -1.0000, -1.0000,  ..., -1.0000, -0.9996, -0.9810],
          [-1.0000, -0.9998, -0.9995,  ..., -0.9999, -0.9998, -1.0000],
          [-1.0000, -1.0000, -0.9994,  ..., -0.9987, -0.9681, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -0.9999, -0.9894, -0.9976],
          [-1.0000, -1.0000, -1.0000,  ..., -0.9998, -1.0000, -0.9870],
          [-1.0000, -1.0000, -0.9999,  ..., -0.9996, -1.0000, -1.0000]]],


        [[[-0.9999, -0.9996, -1.0000,  ..., -1.0000, -1.0000, -0.9999],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -0.9990, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -0.9940, -1.0000, -1.0000]]]],
       grad_fn=<SliceBackward0>)


In [30]:
# Detaching data from CPU/GPU

generated_samples = generated_samples.detach()

In [39]:
# Saving Generated Samples to file
import numpy as np


np.save("generated_samples.npy", generated_samples.numpy())
# for fetching from "generated_samples.npy" file
# arr = np.load('generated_samples.npy')


np.savetxt("generated_samples.txt", generated_samples.numpy().flatten())
# for fetching from "generated_samples.txt" file
# arr = np.loadtxt('generated_samples.txt')
# reshape the array to its original shape
# arr = arr.reshape((2, 3, 4, 5))

In [45]:
torch.Tensor(arr)

tensor([[[[-0.9987, -1.0000, -1.0000,  ..., -1.0000, -0.9996, -0.9810],
          [-1.0000, -0.9998, -0.9995,  ..., -0.9999, -0.9998, -1.0000],
          [-1.0000, -1.0000, -0.9994,  ..., -0.9987, -0.9681, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -0.9999, -0.9894, -0.9976],
          [-1.0000, -1.0000, -1.0000,  ..., -0.9998, -1.0000, -0.9870],
          [-1.0000, -1.0000, -0.9999,  ..., -0.9996, -1.0000, -1.0000]]],


        [[[-0.9999, -0.9996, -1.0000,  ..., -1.0000, -1.0000, -0.9999],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -0.9990, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -0.9940, -1.0000, -1.0000]]],


        [[[-1.0000, -0.9999, -0.9996,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.000

---

## Exporting Trained models 

In [48]:
# Save the generator Model
torch.save(generator_.state_dict(), 'generator_model.pth')

# Save the discriminator Model
torch.save(discriminator_.state_dict(), 'discriminator_model.pth')

In [51]:
# Loading the saved models
gen_model = torch.load('generator_model.pth')
disc_model = torch.load('discriminator_model.pth')

---

## Plotting Generated Samples

In [ ]:
for i in range(16):
    ax = plt.subplot(4,4,i+1)
    plt.imshow(generated_samples[i].reshape(28,28), cmap='gray_r')
    plt.xticks([])
    plt.yticks([])

---